# Getting started

This tutorial shows you how to get started with the `povm_toolbox`.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from povm_toolbox.library.pm_sim_implementation import ClassicalShadows, RandomizedPMs
from povm_toolbox.post_processor.povm_post_processor import POVMPostprocessor
from povm_toolbox.sampler.povm_sampler import POVMSampler
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import (
    Operator,
    SparsePauliOp,
    Statevector,
    random_hermitian,
)
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime import Session

## Random Circuit

We first llok at a random the 2-qubit circuit, with depth 3.

In [3]:
n_qubit = 2
n_cicuits = 5
np.random.seed(14)
seeds= np.random.randint(0,10000000,size=n_cicuits)
qc_random = [random_circuit(num_qubits=n_qubit, depth = 3, measure=False, seed=seed) for seed in seeds]
qc_random[0].draw()

┌────────────┐┌───┐
q_0: ──■──┤ Rx(3.8945) ├┤ I ├
     ┌─┴─┐└─────┬──────┘├───┤
q_1: ┤ Y ├──────■───────┤ X ├
     └───┘              └───┘

We also draw some random observables 

In [4]:
set_observables = [SparsePauliOp.from_operator(random_hermitian(2**n_qubit)) for _ in range(10)]

For reference, we compute the true final state and the exact expectation values for the different observables.

In [5]:
exp_val = np.zeros((len(qc_random), len(set_observables)), dtype=complex)
for i, qc in enumerate(qc_random):
    exact_state = Statevector(np.array(Operator(qc).data[:,0]).squeeze())
    exp_val[i] = np.array([exact_state.expectation_value(obs) for obs in set_observables])
exp_val = np.real_if_close(exp_val)

## Classical Shadows

We now look at the implementation of Classical Shaodws measurement

In [6]:
# By default, the Classical Shadows (CS) measurement uses X,Y,Z measurements with equal probability.
cs_implementation = ClassicalShadows(n_qubit=n_qubit)
# Define the default shot budget.
cs_shots = 4096
cs_implementation.msmt_qc.draw()

┌─────────────────┐┌────┐┌──────────────────┐┌────┐»
            povm_qr_0: ┤ Rz(-1.0*phi[0]) ├┤ √X ├┤ Rz(π - theta[0]) ├┤ √X ├»
                       ├─────────────────┤├────┤├──────────────────┤├────┤»
            povm_qr_1: ┤ Rz(-1.0*phi[1]) ├┤ √X ├┤ Rz(π - theta[1]) ├┤ √X ├»
                       └─────────────────┘└────┘└──────────────────┘└────┘»
povm_measurement_cr: 2/═══════════════════════════════════════════════════»
                                                                          »
«                       ┌────────┐┌─┐   
«            povm_qr_0: ┤ Rz(3π) ├┤M├───
«                       ├────────┤└╥┘┌─┐
«            povm_qr_1: ┤ Rz(3π) ├─╫─┤M├
«                       └────────┘ ║ └╥┘
«povm_measurement_cr: 2/═══════════╩══╩═
«                                  0  1

In [7]:
pubs = []
for qcr in qc_random:
    pubs.append((qcr, None, np.random.randint(1000,10000) if np.random.randint(2)==0 else None, cs_implementation))
print(pubs)

[(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16ab7b070>, None, 2232, <povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x16ab7b1c0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16ab79a20>, None, None, <povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x16ab7b1c0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16ab782e0>, None, 9045, <povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x16ab7b1c0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16ab7a680>, None, 1471, <povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x16ab7b1c0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16ab78f40>, None, 7688, <povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x16ab7b1c0>)]


In [8]:
# Run the sampler job locally using AerSimulator.
# Session syntax is supported but ignored because local mode doesn't support sessions.
backend = AerSimulator()
with Session(backend=backend) as session:
    # First define a standard sampler (that will be used under the hood).
    runtime_sampler = Sampler(session=session)
    # Then define the POVM sampler, which takes BaseSampler as an argument.
    sampler = POVMSampler(runtime_sampler)
    # Submit the job by specifying which POVM to use, which circuit(s) to measure and the shot budget.
    cs_job = sampler.run(pubs, shots=cs_shots)

/Users/dao/Documents/Internship/ic-povms/.venv/lib/python3.10/site-packages/qiskit_ibm_runtime/session.py:157: UserWarning: Session is not supported in local testing mode or when using a simulator.
  warnings.warn(


### Results
We retrieve the result object, which contains the POVM used and from which we can query the counts of each outcome.

In [9]:
cs_result = cs_job.result()
print(f"POVM: {cs_result[0].povm_metadata}")
print(f"Counts: {cs_result[1].get_counts()}")

POVM: RandomizedPMsMetadata(povm=<povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x16ab7b1c0>, pvm_keys=[(2, 1), (1, 0), (1, 2), (0, 0), (2, 1), (0, 1), (2, 2), (1, 0), (2, 0), (2, 0), (2, 1), (0, 2), (0, 0), (1, 1), (2, 1), (0, 1), (2, 2), (1, 2), (0, 1), (0, 0), (2, 0), (0, 0), (2, 2), (0, 0), (2, 0), (2, 1), (1, 1), (2, 2), (1, 2), (0, 2), (1, 2), (2, 1), (1, 1), (0, 1), (0, 0), (2, 0), (0, 2), (1, 2), (2, 0), (2, 0), (0, 1), (0, 2), (0, 2), (0, 1), (2, 0), (0, 0), (0, 0), (1, 0), (1, 1), (2, 2), (1, 0), (1, 2), (0, 0), (1, 2), (0, 2), (0, 0), (1, 2), (1, 0), (2, 2), (0, 0), (0, 0), (0, 1), (1, 1), (1, 2), (1, 2), (0, 2), (1, 2), (1, 2), (0, 2), (1, 2), (0, 1), (0, 0), (2, 2), (0, 1), (2, 0), (1, 2), (0, 1), (1, 0), (1, 2), (1, 0), (2, 1), (2, 1), (1, 0), (1, 0), (1, 2), (2, 0), (1, 0), (0, 2), (1, 2), (2, 1), (0, 0), (1, 0), (0, 1), (2, 2), (0, 2), (0, 2), (0, 0), (0, 0), (0, 0), (2, 1), (2, 1), (2, 2), (0, 1), (1, 0), (2, 0), (1, 2), (0, 0), (1, 2), (2, 2), 

We now define our POVM post-processor, which will use the resukt object to estimate expectation values of some observables.

In [10]:
for i_state in range(len(cs_result)):
    cs_postprocessor = POVMPostprocessor(cs_result[i_state])
    cs_est_exp_val = [cs_postprocessor.get_expectation_value(obs)[0] for obs in set_observables]
    cs_shots = sum(cs_postprocessor.counts[0].values())

    n = int(np.ceil(np.log10(cs_shots)))
    print('Exact        CS')
    print(f'             ({cs_shots} shots)')
    for i in range(len(set_observables)):
        print(f'{np.real(exp_val[i_state,i]):>10.3e}   {cs_est_exp_val[i]:>{8+n}.3e}   {abs(cs_est_exp_val[i]-np.real(exp_val[i_state,i]))/abs(np.real(exp_val[i_state,i])):>{12-n}.1%}')

Exact        CS
             (2232 shots)
-8.353e-01     -8.391e-01       0.5%
 1.117e+00      1.149e+00       2.9%
-5.763e-01     -5.644e-01       2.1%
 2.490e-02      5.574e-02     123.9%
-1.425e+00     -1.402e+00       1.6%
 3.736e-01      4.322e-01      15.7%
 1.316e+00      1.173e+00      10.9%
 1.435e-01      1.697e-01      18.3%
 4.133e-01      4.821e-01      16.6%
-1.032e+00     -1.015e+00       1.7%
Exact        CS
             (4096 shots)
 4.703e-01      5.192e-01      10.4%
 9.608e-01      9.265e-01       3.6%
 1.294e-01      1.772e-01      37.0%
-7.716e-01     -8.257e-01       7.0%
-3.785e-01     -3.659e-01       3.3%
-3.779e-01     -4.529e-01      19.8%
-1.274e-02     -1.049e-01     723.3%
-1.340e+00     -1.242e+00       7.4%
-9.331e-01     -9.504e-01       1.8%
-9.037e-01     -9.502e-01       5.1%
Exact        CS
             (9045 shots)
 2.903e-01      2.342e-01      19.3%
 1.303e+00      1.260e+00       3.3%
-7.930e-01     -8.120e-01       2.4%
-1.726e+00     -1.715e+

## PM-simulable POVM
We now look at POVM that are simulable (through randomization) with only single-qubit projective measurements (PMs).

In [11]:
batch_size = 1000

pubs_pm = []
for pub in pubs:
    pubs_pm.append((pub[0], None, None if pub[2] is None else pub[2]*batch_size, None))

# Define our projective measurements acting on each qubit.
angles = np.array([[0.0, 0.0, 0.5 * np.pi, 0.0, 0.5 * np.pi, 0.5 * np.pi],[0.5 * np.pi, 0.5 * np.pi, 0.0, 0.0, 0.5 * np.pi, 0.0]])#, [0.5 * np.pi, 0.0,0.5 * np.pi, 0.5 * np.pi, 0.0, 0.0]])
# Set the distributions of the shots among the PMs.
bias = np.array([[0.34, 0.33, 0.33], [0.33,0.34,0.33]])#, [0.33,0.33,0.34]])

# Define the PM-simulable POVM.
pmsim_implementation = RandomizedPMs(n_qubit=n_qubit, bias=bias, angles=angles, shot_batch_size=batch_size)

In [12]:
pmsim_shots = 4096*batch_size

with Session(backend=backend) as session:
    # First define a standard sampler (that will be used under the hood).
    runtime_sampler = Sampler(session=session)
    # Then define the POVM sampler, which takes BaseSampler as an argument.
    sampler = POVMSampler(runtime_sampler)
    # Submit the job by specifying which POVM to use, which circuit(s) to measure and the shot budget.
    pmsim_job = sampler.run(pubs_pm, povm=pmsim_implementation, shots=pmsim_shots)

### Results

Retrieve the result of the sampling and use it to estiamte some expectation values via the post-processor

In [13]:
pmsim_result = pmsim_job.result()

for i_state in range(len(pmsim_result)):
    pmsim_postprocessor = POVMPostprocessor(pmsim_result[i_state])
    pmsim_est_exp_val = [pmsim_postprocessor.get_expectation_value(obs)[0] for obs in set_observables]
    pmsim_shots = sum(pmsim_postprocessor.counts[0].values())

    n = int(np.ceil(np.log10(pmsim_shots)))
    print('Exact        PM-sim')
    print(f'             ({pmsim_shots} shots)')
    for i in range(len(set_observables)):
        print(f'{np.real(exp_val[i_state,i]):>10.3e}   {pmsim_est_exp_val[i]:>{8+n}.3e}   {abs(pmsim_est_exp_val[i]-np.real(exp_val[i_state,i]))/abs(np.real(exp_val[i_state,i])):>{12-n}.1%}')

Exact        PM-sim
             (2232000 shots)
-8.353e-01        -8.359e-01    0.1%
 1.117e+00         1.112e+00    0.5%
-5.763e-01        -5.578e-01    3.2%
 2.490e-02         7.563e-03   69.6%
-1.425e+00        -1.410e+00    1.0%
 3.736e-01         3.722e-01    0.4%
 1.316e+00         1.307e+00    0.7%
 1.435e-01         1.467e-01    2.2%
 4.133e-01         3.975e-01    3.8%
-1.032e+00        -1.030e+00    0.2%
Exact        PM-sim
             (4096000 shots)
 4.703e-01         4.696e-01    0.2%
 9.608e-01         9.482e-01    1.3%
 1.294e-01         1.271e-01    1.8%
-7.716e-01        -7.529e-01    2.4%
-3.785e-01        -3.693e-01    2.4%
-3.779e-01        -3.822e-01    1.1%
-1.274e-02        -1.228e-02    3.6%
-1.340e+00        -1.333e+00    0.6%
-9.331e-01        -9.230e-01    1.1%
-9.037e-01        -8.939e-01    1.1%
Exact        PM-sim
             (9045000 shots)
 2.903e-01         2.815e-01    3.0%
 1.303e+00         1.322e+00    1.5%
-7.930e-01        -8.087e-01    2.0%
-1